In [2]:
pip install pyspark


In [3]:
pip install findspark


In [4]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [4]:
spark = SparkSession.builder.appName("student performance").getOrCreate()

In [6]:

dataset = spark.read.csv("/content/Student_performance_data _.csv", inferSchema = True, header = True)
print('There are ',len(dataset.columns),'columns in this dataset')
print('There are ',dataset.count(), 'rows in this dataset')


There are  15 columns in this dataset
There are  2392 rows in this dataset


AttributeError: 'DataFrame' object has no attribute 'isna'

In [ ]:
dataset.show()
# dataset previously cleaned before using so this step is skipped, straight to analysis

+---------+---+------+---------+-----------------+----------------+--------+--------+---------------+---------------+------+-----+------------+-----------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation| StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|              GPA|GradeClass|
+---------+---+------+---------+-----------------+----------------+--------+--------+---------------+---------------+------+-----+------------+-----------------+----------+
|     1001| 17|     1|        0|                2|19.8337228078547|       7|       1|              2|              0|     0|    1|           0| 2.92919559166768|         2|
|     1002| 18|     0|        0|                1|15.4087560558467|       0|       0|              1|              0|     0|    0|           0| 3.04291483343638|         1|
|     1003| 15|     0|        2|                3|4.21056976881226|      26|       0|              2|              0|     0|    0|     

In [ ]:
assembler = VectorAssembler(inputCols = ['Age', 'Gender', 'Ethnicity', 'ParentalEducation', 'StudyTimeWeekly', 'Absences', 'Tutoring', 'ParentalSupport', 'Extracurricular','Sports','Music','Volunteering'], outputCol='features')
output = assembler.transform(dataset)

In [ ]:
finalised_data = output.select('features', 'GradeClass')
finalised_data.show()

+--------------------+----------+
|            features|GradeClass|
+--------------------+----------+
|[17.0,1.0,0.0,2.0...|         2|
|(12,[0,3,4,7],[18...|         1|
|(12,[0,2,3,4,5,7]...|         4|
|[17.0,1.0,0.0,3.0...|         3|
|[17.0,1.0,0.0,2.0...|         4|
|(12,[0,3,4,7,8],[...|         1|
|[15.0,0.0,1.0,1.0...|         2|
|[15.0,1.0,1.0,4.0...|         4|
|(12,[0,4,5,7,9,11...|         2|
|(12,[0,1,3,4,7,8]...|         0|
|(12,[0,3,4,5,7],[...|         3|
|(12,[0,3,4,5,7,11...|         4|
|[17.0,0.0,1.0,1.0...|         4|
|[17.0,0.0,1.0,2.0...|         4|
|[18.0,1.0,0.0,1.0...|         3|
|(12,[0,3,4,5,6,9]...|         4|
|[18.0,0.0,3.0,1.0...|         3|
|(12,[0,1,4,5,6,7]...|         4|
|[18.0,0.0,1.0,3.0...|         4|
|(12,[0,3,4,5,7,10...|         3|
+--------------------+----------+
only showing top 20 rows



In [ ]:
train, test = finalised_data.randomSplit([0.8, 0.2])

In [ ]:
log_reg = LogisticRegression(featuresCol = "features", labelCol="GradeClass")
log_regmodel = log_reg.fit(train)
predictions = log_regmodel.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_eval = MulticlassClassificationEvaluator(labelCol="GradeClass", predictionCol="prediction")
accuracy = multi_eval.evaluate(predictions, {multi_eval.metricName: "accuracy"})
precision = multi_eval.evaluate(predictions, {multi_eval.metricName: "weightedPrecision"})
recall = multi_eval.evaluate(predictions, {multi_eval.metricName: "weightedRecall"})
f1 = multi_eval.evaluate(predictions, {multi_eval.metricName: "f1"})

In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")


Accuracy: 0.7677
Precision: 0.7471
Recall: 0.7677
F1: 0.7500


AttributeError: 'DataFrame' object has no attribute 'rootMeanSquaredError'